In [1]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import warnings
warnings.filterwarnings("ignore", category=UserWarning)  # Ignore UserWarnings for cleaner output
from torchvision import datasets, transforms, models

# Define transforms for your dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])test
])

# Load local custom dataset (assumes folder structure: root/class_x/xxx.png)
train_data = datasets.ImageFolder(root='data/train', transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
train_data.classes

['0__Anger',
 '1__Contempt',
 '2__Disgust',
 '3__Fear',
 '4__Happy',
 '5__Neutral',
 '6__Sad',
 '7__Surprise']

In [2]:

# Initialize the model, loss function, and optimizer
model = nn.Sequential(
    models.resnet18(pretrained=True),
    nn.Linear(1000, 512),
    nn.ReLU(),
    nn.Linear(512, 8),
    nn.LogSoftmax(dim=1)
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/ilikesunday/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100.0%


In [3]:
epochs = 20
try:

# Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device),labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')
        torch.save(model.state_dict(), 'face_expression_model.pth')
    
except Exception as e:
    print(f"An error occurred during training: {e}")
finally:
    with open('Emotion_classes.txt', 'w') as f:
        for cls in train_data.classes:
            f.write(f"{cls}\n")
    print("Training completed or interrupted.")
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

Epoch [1/20], Loss: 1.3754
Epoch [2/20], Loss: 1.0899
Epoch [3/20], Loss: 0.9333
Epoch [4/20], Loss: 0.8027
Epoch [5/20], Loss: 0.6876
Epoch [6/20], Loss: 0.5636
Epoch [7/20], Loss: 0.4464
Epoch [8/20], Loss: 0.3366
Epoch [9/20], Loss: 0.2787
Epoch [10/20], Loss: 0.2040
Epoch [11/20], Loss: 0.1842
Epoch [12/20], Loss: 0.1479
Epoch [13/20], Loss: 0.1574
Epoch [14/20], Loss: 0.1262
Epoch [15/20], Loss: 0.1070
Epoch [16/20], Loss: 0.1119
Epoch [17/20], Loss: 0.1338
Epoch [18/20], Loss: 0.1037
Epoch [19/20], Loss: 0.0946
Epoch [20/20], Loss: 0.0870
Training completed or interrupted.


In [7]:

test_data = datasets.ImageFolder(root='data/test', transform=transform)
test_loader = DataLoader(train_data, batch_size=32, shuffle=False)
val_dataset = datasets.ImageFolder(root='data/valid', transform=transform)
val_loader = DataLoader(train_data, batch_size=32, shuffle=False)
def model_accuracy(model:nn.Sequential, data_loader:DataLoader,data_type:str)->int:
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'{data_type} set accuracy: {accuracy:.2f}%')
    return round(accuracy,2)


print(model_accuracy(model, test_loader, data_type="Test"))
print(model_accuracy(model, val_loader, data_type="Validation"))


Test set accuracy: 97.26%
97.26
Validation set accuracy: 97.26%
97.26


In [8]:
# Save the model
torch.save(model.state_dict(), 'face_expression_model.pth')
# Save the class labels
import pickle
with open("Emotions.pkl", 'wb') as f:
    pickle.dump(train_data.classes, f)
print(f"Saved {len(train_data.classes)} class labels to 'Emotions.pkl'")

Saved 8 class labels to 'Emotions.pkl'
